<a href="https://colab.research.google.com/github/memona008/Deep-Learning-Notebooks/blob/master/Dogs_vs_Cats_AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading Kaggle Dogs vs cats dataset)

In [ ]:
#upload your kaggle.json file here
from google.colab import files
files.upload()

In [ ]:
#downloading the dataset
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

!kaggle kernels list — user memonasultan — sort-by dateRun

!kaggle datasets download -d biaiscience/dogs-vs-cats



kaggle.json
usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: unrecognized arguments: — user memonasultan — sort-by dateRun
 98% 801M/817M [00:16<00:00, 45.2MB/s]
100% 817M/817M [00:16<00:00, 51.6MB/s]


In [ ]:
#unzipping the dataset

!unzip -q dogs-vs-cats.zip -d .
!ls

dogs-vs-cats.zip  drive  kaggle.json  sample_data  test  train


#AlexNet

In [ ]:
!pip install keras

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2
import random
import keras
from keras import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from tokenize import tokenize
from os import makedirs
from os import listdir
from shutil import copyfile

Using TensorFlow backend.


In [ ]:
training_set_path = "./train/train"
testing_set_path = "./test/test"
CATEGORIES = ['cat','dog']

#for recreating the data set for ImageGenerator.flow_from_directory :P 
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']

In [ ]:
for subdir in subdirs:
	# create label subdirectories
	labeldirs = ['dogs/', 'cats/']
	for labldir in labeldirs:
		newdir = dataset_home + subdir + labldir
		makedirs(newdir, exist_ok=True)

In [ ]:
# making structure 
src_directory = training_set_path
for file in listdir(src_directory):
	src = src_directory + '/' + file
	dst_dir = 'train/'
	if file.startswith('cat'):
		dst = dataset_home + dst_dir + 'cats/'  + file
		copyfile(src, dst)
	elif file.startswith('dog'):
		dst = dataset_home + dst_dir + 'dogs/'  + file
		copyfile(src, dst)
  



In [ ]:
TRAIN_DIR = dataset_home + 'train/'
RESIZE_SHAPE = (224,224) #227 227 for alexNet

In [ ]:
train_datagen = preprocessing.image.ImageDataGenerator(validation_split=0.2,rescale=1. / 255)
train_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=RESIZE_SHAPE,
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical", 
    subset = 'training',
    seed=42
)

valid_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=RESIZE_SHAPE,
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical", 
    subset = 'validation',
    seed=42
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
def get_AlexNet_model():
  model = Sequential()
  # 1st layer
  conv_layer_1 = Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu')
  model.add(conv_layer_1)
  model.add(BatchNormalization(axis = -1))
  max_pool_1 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid')
  model.add(max_pool_1)


  # 2nd layer
  conv_layer_2 = Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid', activation='relu')
  model.add(conv_layer_2)
  model.add(BatchNormalization(axis = -1))
  max_pool_2 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid')
  model.add(max_pool_2)

   # 3rd layer
  conv_layer_3 = Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu')
  model.add(conv_layer_3)
  model.add(BatchNormalization(axis = -1))

  # 4th layer
  conv_layer_4 = Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu')
  model.add(conv_layer_4)
  model.add(BatchNormalization(axis = -1))

  # 5th layer
  conv_layer_5 = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu')
  model.add(conv_layer_5)
  model.add(BatchNormalization(axis = -1))
  max_pool_5 = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid')
  model.add(max_pool_5)

  model.add(Flatten())

  #Fully Connected Layers
  model.add(Dense(4096, input_shape=(227,227,3), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))
  model.add(Dense(4096, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))
  model.add(Dense(1000, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))
  model.add(Dense(len(CATEGORIES), activation='softmax'))

  return model


In [ ]:
model = get_AlexNet_model()
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization_1 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 256)       614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 23, 23, 256)       1024      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 384)        

In [ ]:
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath='./model/weights.{epoch:02d}-{val_accuracy:.2f}.hdf5', 
    verbose=1, 
    monitor='val_accuracy',
    mode = 'max',
    save_weights_only=False, save_best_only=True )

model.fit_generator(generator=train_generator,steps_per_epoch=20000/32, callbacks=[cp_callback],
	validation_data=valid_generator, validation_steps=5000/32 , epochs=100, verbose=1)

Epoch 1/100
625/625 [==============================] - 100s 160ms/step - loss: 0.1808 - accuracy: 0.9276 - val_loss: 0.1396 - val_accuracy: 0.8760

Epoch 00001: val_accuracy improved from -inf to 0.87600, saving model to ./model/weights.01-0.88.hdf5
Epoch 2/100
625/625 [==============================] - 98s 157ms/step - loss: 0.1604 - accuracy: 0.9366 - val_loss: 0.2025 - val_accuracy: 0.8490

Epoch 00002: val_accuracy did not improve from 0.87600
Epoch 3/100
625/625 [==============================] - 98s 157ms/step - loss: 0.1408 - accuracy: 0.9452 - val_loss: 0.2123 - val_accuracy: 0.8276

Epoch 00003: val_accuracy did not improve from 0.87600
Epoch 4/100
625/625 [==============================] - 99s 158ms/step - loss: 0.1295 - accuracy: 0.9501 - val_loss: 1.4696 - val_accuracy: 0.8336

Epoch 00004: val_accuracy did not improve from 0.87600
Epoch 5/100
625/625 [==============================] - 98s 157ms/step - loss: 0.1110 - accuracy: 0.9597 - val_loss: 0.9672 - val_accuracy: 0.871

#Prediction with restored AlexNet best model saved while training (90% accuracy)

In [ ]:
restored_model = get_AlexNet_model()
restored_model.load_weights("/content/drive/My Drive/Models/weights.56-0.90.hdf5")
restored_model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])

In [ ]:
restored_model.summary()

In [ ]:
def load_image(filename):
	# load the image
	img = keras.preprocessing.image.load_img(filename, target_size=IMAGE_SHAPE, color_mode='rgb')
	# convert to array
	img = keras.preprocessing.image.img_to_array(img)
	return img

In [ ]:
def preprocess_image(img):
  img = img.reshape(1,224, 224, 3)
  return img

In [ ]:
def predict_samples(model,from_n=0):
  plt.figure(figsize=(25,25))
  for i in range(1,25):
      filename = f'{testing_set_path}/{i+from_n}.jpg'
      img = load_image(filename)
      plt.subplot(5,5,i+1)
      plt.xticks([])
      plt.yticks([])
      plt.grid(False)
      plt.imshow(keras.preprocessing.image.load_img(filename, color_mode='rgb'))
      predict = model.predict(preprocess_image(img))
      prediction = np.argmax(predict, axis=1)
      plt.rcParams.update({'font.size': 20}) 
      plt.xlabel(CATEGORIES[prediction[0]])
  plt.show()